Iremos utilizar a biblioteca SpaCy no lugar do NLTK e criar um modelo Word2Vec do zero sem uma base pré-treinada, assim você vai conseguir criar um classificador com o seu próprio modelo.

# SpaCy

**SpaCy** é uma biblioteca de processamento de linguagem natural (NLP) de código aberto para Python. É amplamente utilizada para construir aplicações que processam e "compreendem" textos escritos, como análise de sentimento, reconhecimento de entidades nomeadas (NER), segmentação de frases, lematização, e muito mais. SpaCy é conhecida por ser rápida, eficiente e fácil de usar.

## Características principais do spaCy

1. **Desempenho**: Projetada para ser eficiente e rápida, spaCy é capaz de processar grandes volumes de texto com rapidez, tornando-a adequada para aplicações em produção.

2. **Modelos pré-treinados**: Oferece uma série de modelos pré-treinados para várias línguas, o que facilita a implementação de tarefas comuns de NLP sem a necessidade de treinamento manual de modelos.

3. **Funcionalidades avançadas**: Suporta uma ampla gama de tarefas de NLP, incluindo:
   - Tokenização
   - POS tagging (marcação de partes do discurso)
   - Lematização
   - Reconhecimento de entidades nomeadas (NER)
   - Análise de dependência sintática
   - Vetores de palavras (word vectors)

4. **Extensibilidade**: Permite a integração com outras bibliotecas de aprendizado de máquina e NLP, como TensorFlow, PyTorch, e scikit-learn, facilitando a criação de pipelines complexas de processamento de texto.

5. **Comunidade e suporte**: Tem uma comunidade ativa e uma boa documentação, com tutoriais e exemplos que ajudam novos usuários a começar rapidamente.

## Resumo

SpaCy é uma poderosa biblioteca de NLP que oferece ferramentas eficientes e robustas para processamento de texto, com suporte a modelos pré-treinados e integração com outras tecnologias de aprendizado de máquina, tornando-a uma escolha popular entre desenvolvedores e pesquisadores.


# Inicio

In [1]:
import pandas as pd

artigo_treino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Word2Vec/treino.csv')
artigo_teste = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Word2Vec/teste.csv')

## Instalação do spacy

https://spacy.io/usage

pip install -U pip setuptools wheel (obs: já instalado no colab)<br/>
pip install -U spacy (obs: já instalado no colab)<br/>
python -m spacy download pt_core_news_sm (obs: pacote em portugues)

In [4]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 35.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


OBS:  após baixar o pacote irá ter a seguinte mesangem, "Restart to reload dependencie". Para fazer isso ir em "Ambiente de execução" -> "Reiniciar sessão"

In [2]:
import spacy
nlp = spacy.load('pt_core_news_sm')

In [4]:
texto = 'Adoro a cidade de caldas novas!'
doc = nlp(texto)

In [5]:
type(doc)

spacy.tokens.doc.Doc

In [6]:
doc.ents

(Adoro, caldas novas)

In [7]:
doc[1].is_stop

True

In [9]:
doc[0].is_stop

False

In [10]:
doc[1].is_alpha

True

In [11]:
def trata_textos(doc):
  tokens_validos = []

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)

  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

In [12]:
texto = 'Adoro a 23424342 #$#$#$#$#$ cidade de caldas novas!'
doc = nlp(texto)
trata_textos(doc)

'Adoro cidade caldas'

In [17]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

# batch_size -> tamanho do bloco de processamento
# n_process -> quantidade de núcleos do processador
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                         batch_size=1000,
                                                         n_process=-1)]

In [25]:
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


## CBOW

In [19]:
from gensim.models import Word2Vec

# sg - 0 -> cbow - 1 -> skipgram
# window - quantidade de palavras antes e depois da palavra central
# vector_size - tamanho do vetor a ser gerado
# min_count - numero de vezes que uma palavra deve aparecer
# alpha - taxa de aprendizagem da rede neural
# min_alpha - mínimo de convergência da rede neural
w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.007)

In [26]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [27]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [28]:
w2v_modelo.build_vocab(lista_lista_tokens)

## Treinando o modelo

In [29]:
# lista todas as propriedas e métodos
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [30]:
# quantidade de corpus textual
w2v_modelo.corpus_count

84466

In [31]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
      print('Loss após a época {}:{}'.format(self.epoch, loss))
    else:
      print('Loss após a época {}:{}'.format(self.epoch, loss - self.loss_previus_step))
    self.epoch += 1
    self.loss_previus_step = loss

In [33]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 compute_loss=True,
                 callbacks=[callback()])

Loss após a época 0:630808.8125
Loss após a época 1:488331.4375
Loss após a época 2:385408.375
Loss após a época 3:330840.625
Loss após a época 4:324775.0
Loss após a época 5:296352.25
Loss após a época 6:292798.25
Loss após a época 7:306675.5
Loss após a época 8:260607.75
Loss após a época 9:258027.75
Loss após a época 10:231077.5
Loss após a época 11:220652.0
Loss após a época 12:235768.75
Loss após a época 13:208946.5
Loss após a época 14:171484.5
Loss após a época 15:181726.5
Loss após a época 16:165515.5
Loss após a época 17:150986.5
Loss após a época 18:152752.5
Loss após a época 19:139693.5
Loss após a época 20:119298.0
Loss após a época 21:115141.0
Loss após a época 22:133518.0
Loss após a época 23:123292.5
Loss após a época 24:112390.5
Loss após a época 25:109408.5
Loss após a época 26:119272.0
Loss após a época 27:110278.5
Loss após a época 28:107891.5
Loss após a época 29:111863.5


(14584049, 16207260)

In [34]:
w2v_modelo.wv.most_similar('google')

[('apple', 0.5859817266464233),
 ('facebook', 0.535375714302063),
 ('airbnb', 0.49373987317085266),
 ('waze', 0.4825575351715088),
 ('uber', 0.47694164514541626),
 ('amazon', 0.4559515416622162),
 ('fbi', 0.43990829586982727),
 ('software', 0.4390987455844879),
 ('samsung', 0.4358971416950226),
 ('tesla', 0.4357891082763672)]

## Skip Gram

In [36]:
# o skip gram tabralha melhor com mais dados, então nesse caso seria melhor aumentar o valor do window de 2 para 5
w2v_modelo_sg = Word2Vec(sg=1,
                      window=5,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 compute_loss=True,
                 callbacks=[callback()])

Loss após a época 0:1881584.0
Loss após a época 1:1441888.75
Loss após a época 2:1185785.25
Loss após a época 3:1020172.5
Loss após a época 4:1045340.0
Loss após a época 5:1063756.5
Loss após a época 6:996801.0
Loss após a época 7:876929.0
Loss após a época 8:807567.0
Loss após a época 9:783224.0
Loss após a época 10:850364.0
Loss após a época 11:785381.0
Loss após a época 12:683849.0
Loss após a época 13:706468.0
Loss após a época 14:687924.0
Loss após a época 15:709081.0
Loss após a época 16:622009.0
Loss após a época 17:695248.0
Loss após a época 18:475296.0
Loss após a época 19:488956.0
Loss após a época 20:455742.0
Loss após a época 21:474158.0
Loss após a época 22:438262.0
Loss após a época 23:431332.0
Loss após a época 24:446956.0
Loss após a época 25:418222.0
Loss após a época 26:432944.0
Loss após a época 27:402192.0
Loss após a época 28:419874.0
Loss após a época 29:391202.0


(14584473, 16207260)

In [37]:
w2v_modelo_sg.wv.most_similar('google')

[('android', 0.385531485080719),
 ('reguladores', 0.3840128481388092),
 ('facebook', 0.3777120113372803),
 ('apple', 0.3689356744289398),
 ('waze', 0.3646523058414459),
 ('buffett', 0.35657042264938354),
 ('anúncios', 0.349619597196579),
 ('concorda', 0.3482460081577301),
 ('patentes', 0.34747374057769775),
 ('anunciantes', 0.3468451201915741)]

# Exportando modelos

In [41]:
w2v_modelo.wv.save_word2vec_format('/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_sg.txt', binary=False)

# Criando o classificador com o nosso modelo

In [60]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras_numeros, modelo):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante =+ modelo.wv.get_vector(pn)
    except KeyError:
      pass
  return vetor_resultante

In [54]:
def tokenizador(texto):
  tokens_validos = []
  doc = nlp(texto)

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

In [58]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)
  return matriz

In [61]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo)

matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

In [62]:
print(matriz_vetores_treino_cbow.shape)

(90000, 300)


In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter=800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL

In [65]:
RL_cbow = classificador(matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.59      0.39      0.47      6103
   cotidiano       0.34      0.56      0.43      1698
     esporte       0.69      0.60      0.64      4663
   ilustrada       0.04      0.56      0.07       131
     mercado       0.67      0.57      0.62      5867
       mundo       0.45      0.60      0.51      2051

    accuracy                           0.53     20513
   macro avg       0.46      0.55      0.46     20513
weighted avg       0.60      0.53      0.55     20513



In [66]:
RL_sg = classificador(matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.63      0.36      0.46      6103
   cotidiano       0.34      0.58      0.43      1698
     esporte       0.74      0.59      0.66      4663
   ilustrada       0.04      0.62      0.08       131
     mercado       0.67      0.61      0.64      5867
       mundo       0.42      0.64      0.51      2051

    accuracy                           0.53     20513
   macro avg       0.47      0.57      0.46     20513
weighted avg       0.62      0.53      0.56     20513

